In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config InlineBackend.figure_format = 'retina'
from setup import *

Extracting data from ../Data/behavior_free_swimming.tar.gz...
Extracting models from ../Models/hmms_20240125.tar.gz...
Extracting models from ../Models/longtrajectories_20240202.tar.gz...


# Figure

# Supplementaries